Experiment with diff. MCP Servers

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

True

The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.
It's a persistent memory store of entities, observations about them, and relationships between them.
https://github.com/modelcontextprotocol/servers/tree/main/src/memory

In [2]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/gulnara.db"}}

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

mcp_tools


[Tool(name='create_entities', description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, annotations=None),
 Tool(name='search_nodes', description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, annotations=None),
 Tool(name='read_graph', description='Get 

In [3]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = "My name's Gulnara. I am a results-driven Software Engineer with 10+ years in software engineering and 5+ years of hands-on experience in applied machine learning and AI-driven application development. Proven record of delivering scalable, production-ready solutions using deep learning, XGBoost, and LLMs—highlighted by a patented chromatographic peak classifier achieving 95% accuracy. Adept in MLOps, cloud-native architectures, and engineering best practices to drive digital transformation. Known for reliability, problem-solving, and leading cross-functional teams. Seeking to contribute ML expertise to mission-driven organizations."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))



Nice to meet you, Gulnara! With your extensive experience in software engineering, machine learning, and AI-driven application development, plus your proven achievements, you have a strong profile. How can I assist you further today?

In [4]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Gulnara. What do you know about me?")
    display(Markdown(result.final_output))

Gulnara is a results-driven Software Engineer with over 10 years of experience in software engineering. She has more than 5 years of hands-on experience in applied machine learning and AI-driven application development. She has a proven record of delivering scalable, production-ready solutions using deep learning, XGBoost, and large language models (LLMs). 

Gulnara created a patented chromatographic peak classifier achieving 95% accuracy. She is adept in MLOps, cloud-native architectures, and engineering best practices to drive digital transformation. She is known for reliability, problem-solving skills, and leading cross-functional teams. Gulnara is seeking to contribute her machine learning expertise to mission-driven organizations. 

Would you like me to add or update any information about you?

Check the trace: https://platform.openai.com/traces

COMMON: The 2nd type of MCP server - runs locally, calls a web service

Using Brave Search

In [5]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools=await server.list_tools()

mcp_tools

[Tool(name='brave_web_search', description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, annotations=None),
 Tool(name='brave_local_search', description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, restaurants, services, etc. Returns detailed information including:\

In [6]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Palantir stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [7]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

As of August 21, 2025, Palantir Technologies (NASDAQ: PLTR) is experiencing a significant increase in its stock price and positive momentum:

1. **Current Performance**: Palantir shares have seen a remarkable rise, gaining over 148% in 2025 alone. The stock recently reached an all-time high of **$190.00** on August 12, 2025.

2. **Recent Trends**: Over the past month, the stock has increased by 31.40%, and it has gained **535.53%** over the last year, indicating strong investor confidence and interest.

3. **Earnings Growth**: Analysts are optimistic about the company’s future earnings. For instance, Palantir's earnings per share (EPS) rose from 8 cents in Q1 2024 to **13 cents** in Q1 2025, marking a substantial increase of 62.5%.

4. **Outlook**: Despite some concerns regarding valuation, many analysts suggest continued growth due to increasing earnings, making Palantir an attractive option for investors looking for growth in the tech sector.

Overall, Palantir's stock is positioned strongly in the market with optimistic projections for the future, driven by its robust performance and earnings growth.

The third type: running remotely

It's actually really hard to find a "remote MCP server" aka "hosted MCP server" aka "managed MCP server".

It's not a common model for using or sharing MCP servers, and there isn't a standard way to discover remote MCP servers.

Anthropic lists some remote MCP servers, but these are for paid applications with business users:

https://docs.anthropic.com/en/docs/agents-and-tools/remote-mcp-servers

CloudFlare has tooling for you to create and deploy your own remote MCP servers, but this does not seem to be a common practice:

https://developers.cloudflare.com/agents/guides/remote-mcp-server/

Trying the Polygon.io MCP Server

In [8]:
load_dotenv(override=True)
polygon_api_key = os.getenv("POLYGON_API_KEY")

In [9]:
from polygon import RESTClient
client = RESTClient(polygon_api_key)
client.get_previous_close_agg("AAPL")[0]

PreviousCloseAgg(ticker='AAPL', close=226.01, high=230.47, low=225.77, open=229.98, timestamp=1755720000000, volume=42263865.0, vwap=226.7667)

created our own MCP server with free version

In [10]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, annotations=None)]

In [11]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Apple (AAPL) is $226.01.

For paid version, very easy to use tools


In [12]:
params = {"command": "uvx",
          "args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"],
          "env": {"POLYGON_API_KEY": polygon_api_key}
          }
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='get_aggs', description='\n    List aggregate bars for a ticker over a given date range in custom time window sizes.\n    ', inputSchema={'properties': {'ticker': {'title': 'Ticker', 'type': 'string'}, 'multiplier': {'title': 'Multiplier', 'type': 'integer'}, 'timespan': {'title': 'Timespan', 'type': 'string'}, 'from_': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'From'}, 'to': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'format': 'date-time', 'type': 'string'}, {'format': 'date', 'type': 'string'}], 'title': 'To'}, 'adjusted': {'anyOf': [{'type': 'boolean'}, {'type': 'null'}], 'default': None, 'title': 'Adjusted'}, 'sort': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Sort'}, 'limit': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Limit'}, 'params': {'anyOf': [{'additionalProperties': True, 'type': 'object'

In [13]:
instructions = "You answer questions about the stock market."
request = "What's the share price of Apple? Use your get_snapshot_ticker tool to get the latest price."
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

I currently don't have authorized access to get the latest share price of Apple (AAPL) using the get_snapshot_ticker tool. Is there any other way I can assist you?